In [1]:
"""
Preparing model:
 - Install bazel ( check tensorflow's github for more info )
    Ubuntu 14.04:
        - Requirements:
            sudo add-apt-repository ppa:webupd8team/java
            sudo apt-get update
            sudo apt-get install oracle-java8-installer
        - Download bazel, ( https://github.com/bazelbuild/bazel/releases )
          tested on: https://github.com/bazelbuild/bazel/releases/download/0.2.0/bazel-0.2.0-jdk7-installer-linux-x86_64.sh
        - chmod +x PATH_TO_INSTALL.SH
        - ./PATH_TO_INSTALL.SH --user
        - Place bazel onto path ( exact path to store shown in the output)
- For retraining, prepare folder structure as
    - root_folder_name
        - class 1
            - file1
            - file2
        - class 2
            - file1
            - file2
- Clone tensorflow

Training:
- Go to root of tensorflow
bazel build -c opt --copt=-mavx tensorflow/examples/image_retraining:retrain
bazel-bin/tensorflow/examples/image_retraining/retrain --image_dir /home/ascend/Documents/perception/AscendDNN/DatabaseCreation/NewDatabase/Training --output_graph /home/ascend/Documents/perception/alexandria/output_graph.pb --output_labels /home/ascend/Documents/perception/alexandria/output_labels.txt --bottleneck_dir /home/ascend/Documents/perception/alexandria/bottleneck

Run tensorboard
python tensorflow/tensorboard/tensorboard.py --logdir /home/ascend/Documents/perception/alexandria

Running:
bazel build tensorflow/examples/label_image:label_image && \
bazel-bin/tensorflow/examples/label_image/label_image \
--graph=/home/ascend/Documents/perception/alexandria/output_graph.pb --labels=/home/ascend/Documents/perception/alexandria/output_labels.txt \
--output_layer=final_result \
--image=/home/ascend/Documents/perception/data/green/2016-10-09-210940.jpg


For testing through python, change and run this code.
"""

"\nPreparing model:\n - Install bazel ( check tensorflow's github for more info )\n    Ubuntu 14.04:\n        - Requirements:\n            sudo add-apt-repository ppa:webupd8team/java\n            sudo apt-get update\n            sudo apt-get install oracle-java8-installer\n        - Download bazel, ( https://github.com/bazelbuild/bazel/releases )\n          tested on: https://github.com/bazelbuild/bazel/releases/download/0.2.0/bazel-0.2.0-jdk7-installer-linux-x86_64.sh\n        - chmod +x PATH_TO_INSTALL.SH\n        - ./PATH_TO_INSTALL.SH --user\n        - Place bazel onto path ( exact path to store shown in the output)\n- For retraining, prepare folder structure as\n    - root_folder_name\n        - class 1\n            - file1\n            - file2\n        - class 2\n            - file1\n            - file2\n- Clone tensorflow\n\nTraining:\n- Go to root of tensorflow\nbazel build -c opt --copt=-mavx tensorflow/examples/image_retraining:retrain\nbazel-bin/tensorflow/examples/image_re

In [1]:
import numpy as np
import tensorflow as tf
import time
import cv2
from threading import Thread
from time import sleep
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
from math import sin, cos, radians
import random
#%matplotlib inline

class Network:

    #ef __init__(self, model, inputWidth, inputHeight, classes, load_preTrained_weights=True, \
    #   trained_w_epochs=0, trained_w_batch_size=0, trained_on_dataset="empty_set", program_version=0):
    def __init__(self):

        self.modelFullPath = '/home/ascend/Documents/perception/alexandria/output_graph.pb'
        self.labelsFullPath = '/home/ascend/Documents/perception/alexandria/output_labels.txt'

        self.full_image = np.asarray([])
        self.input_image = np.asarray([])
        self.found_targets_bounding_boxes = []

        self.stoppedPropagationThread = False
        
        # Creates graph from saved GraphDef.
        self.create_graph()

        self.sess = tf.Session()
        self.f = open(self.labelsFullPath, 'rb')
        self.lines = self.f.readlines()
        self.labels = [str(w).replace("\n", "") for w in self.lines]
        
        # assumtion of the world
        self.ystart = self.get_ystart()
        #self.ystart = 100
        self.yend = self.get_yend()
        self.ys = [y for y in range(self.ystart,self.yend+1)]
        self.robotRadiuses = [self.pixel2robotRadius(y) for y in range(self.ystart,self.yend+1)]

        print "network initialized"
        

    def create_graph(self):
        """Creates a graph from saved GraphDef file and returns a saver."""
        # Creates graph from saved graph_def.pb.
        with tf.gfile.FastGFile(self.modelFullPath, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='')

    def run_inference_on_video(self):
        answer = None
        softmax_tensor = self.sess.graph.get_tensor_by_name('final_result:0')
        predictions = self.sess.run(softmax_tensor, {'DecodeJpeg:0': self.input_image})
        predictions = np.squeeze(predictions)
        top_k = predictions.argsort()[-2:][::-1]  # Getting top 5 predictions

        #for node_id in top_k:
        #    human_string = labels[node_id]
        #    score = predictions[node_id]
        #    print('%s (score = %.5f)' % (human_string, score)),
        #print "\n"
        #answer = self.labels[top_k[0]]
        answer = (self.labels[top_k[0]], predictions[top_k[0]])
        return answer
    
    def pixel2scale(self, height, angle, pixel_y):
        # returns number of meters one pixel width is on the ground
        # F.ex if it returns 2, 10 pixels width is 20 meters on the ground

        # Will return negative values if the pixel does not correspond to a point on the ground

        # height from ground to camera in meters
        # angle from downwards vector to camera center direction
        # pixel y coordinate from top

        #pinhole camera model
        camera_ox = 640/2
        camera_oy = 480/2
        camera_focal = 1360.4/1080 * 480    

        dy = pixel_y-camera_oy
        dz = camera_focal

        sa, ca = sin(radians(angle)), cos(radians(angle))
        dz = ca*dz+sa*dy

        t = height/dz
        return t

    def pixel2robotRadius(self, y):
        #return 0.33/self.pixel2scale(1, 70, y)
        return 0.30/self.pixel2scale(0.65, 60, y)

    def get_ystart(self):
        smallest_i = None
        for i in range(480):
            if self.pixel2robotRadius(i) > 0 and smallest_i == None:
                smallest_i = i
        return smallest_i

    def get_yend(self):
        yend = None
        for y in range(self.ystart, 480):
            if y + self.pixel2robotRadius(y) < 480:
                yend = y
        return yend
    
    def feed_full_image(self, img):
        # non-blocking operation
        self.full_image = img
    
    def search_image_for_targets(self):
        # per pixel
        #self.found_targets_bounding_boxes = []
        if self.full_image.size != 0:
            print "searching image"
            pixel_width = len(self.full_image[0,:])
            # blocking operation
            for y, r in zip(self.ys,self.robotRadiuses):
                #x_steps = int(pixel_width/r)
                for x in range(0, pixel_width - int(r)):
                    crop = self.full_image[y:y+r,x:x+r]
                    self.input_image = cv2.resize(crop, (64,64))
                    #if self.run_inference_on_video() == 'robot':
                    (label, pred) = self.run_inference_on_video()
                    if label == 'robot' and pred > 0.9:
                        x1 = x
                        y1 = y
                        x2 = int(x+r)
                        y2 = int(y+r)
                        #if not (x1,y1,x2,y2) in self.found_targets_bounding_boxes:
                        #    self.found_targets_bounding_boxes.append((x1,y1,x2,y2))
                        self.found_targets_bounding_boxes.append((x1,y1,x2,y2))

    def search_image_for_targets_2(self):
        # using beta function on y axis
        if self.full_image.size != 0:
            print "searching image"
            pixel_width = len(self.full_image[0,:])
            # blocking operation
            
            
            i = int(random.betavariate(6,1.2) * (len(self.ys)-1))
            y = self.ys[i]
            r = int(self.robotRadiuses[i])
            print "chose y: %i" % y
            print "chose r: %i" % r
            
            x_steps = int(pixel_width/r)
            
            #for x in range(0, pixel_width - int(r)):
            for x in range(x_steps):
                print "looking at x: %i y: %i with radius: %i" % (x*r, y, r)
                #print "looking at x: %i y: %i with radius: %i" % (x, y, r)
                #crop = self.full_image[y:y+r,x:x+r]
                
                crop = self.full_image[y:y+r,x*r:x*r+r]
                
                self.input_image = cv2.resize(crop, (64,64))
                #if self.run_inference_on_video() == 'robot':
                (label, pred) = self.run_inference_on_video()
                if label == 'robot' and pred > 0.9:
                    #x1 = x
                    x1 = x*r
                    y1 = y
                    #x2 = int(x+r)
                    x2 = int(x*r+r)
                    y2 = int(y+r)
                    if not (x1,y1,x2,y2) in self.found_targets_bounding_boxes:
                        self.found_targets_bounding_boxes.append((x1,y1,x2,y2))
                        
    def search_image_for_targets_3(self):
        # non-overlapping sliding window
        if self.full_image.size != 0:
            print "searching image"
            pixel_width = len(self.full_image[0,:])
            # blocking operation
            
            
            #i = int(random.betavariate(6,1.2) * (len(self.ys)-1))
            #y = self.ys[i]
            #r = int(self.robotRadiuses[i])
            #print "chose y: %i" % y
            #print "chose r: %i" % r
            
            y = self.ystart
            while True:
                r = int(self.pixel2robotRadius(y))
                if y + r > 480:
                    break

                x_steps = int(pixel_width/r)

                for x in range(x_steps):
                    print "looking at x: %i y: %i with radius: %i" % (x*r, y, r)
                    #print "looking at x: %i y: %i with radius: %i" % (x, y, r)
                    #crop = self.full_image[y:y+r,x:x+r]

                    crop = self.full_image[y:y+r,x*r:x*r+r]

                    self.input_image = cv2.resize(crop, (64,64))
                    #if self.run_inference_on_video() == 'robot':
                    (label, pred) = self.run_inference_on_video()
                    if label == 'robot' and pred > 0.9:
                        #x1 = x
                        x1 = x*r
                        y1 = y
                        #x2 = int(x+r)
                        x2 = int(x*r+r)
                        y2 = int(y+r)
                        if not (x1,y1,x2,y2) in self.found_targets_bounding_boxes:
                            self.found_targets_bounding_boxes.append((x1,y1,x2,y2))
                                        
                y += r
                
    def search_image_for_targets_4(self):
        # uniformly random y
        if self.full_image.size != 0:
            print "searching image"
            pixel_width = len(self.full_image[0,:])
            # blocking operation
            
            
            i = int(random.randrange(1) * (len(self.ys)-1))
            y = self.ys[i]
            r = int(self.robotRadiuses[i])
            print "chose y: %i" % y
            print "chose r: %i" % r
            
            x_steps = int(pixel_width/r)
            
            #for x in range(0, pixel_width - int(r)):
            for x in range(x_steps):
                print "looking at x: %i y: %i with radius: %i" % (x*r, y, r)
                #print "looking at x: %i y: %i with radius: %i" % (x, y, r)
                #crop = self.full_image[y:y+r,x:x+r]
                
                crop = self.full_image[y:y+r,x*r:x*r+r]
                
                self.input_image = cv2.resize(crop, (64,64))
                #if self.run_inference_on_video() == 'robot':
                (label, pred) = self.run_inference_on_video()
                if label == 'robot' and pred > 0.9:
                    #x1 = x
                    x1 = x*r
                    y1 = y
                    #x2 = int(x+r)
                    x2 = int(x*r+r)
                    y2 = int(y+r)
                    if not (x1,y1,x2,y2) in self.found_targets_bounding_boxes:
                        self.found_targets_bounding_boxes.append((x1,y1,x2,y2))

    def startPropagationThread(self):
        t = Thread(target=self.propagateThread, args=())
        t.daemon = True
        print "starting propagation thread"
        t.start()
        return self

    def propagateThread(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stoppedPropagationThread:
                return
            # otherwise, propagate
            self.search_image_for_targets_3()

    def stopPropagationThread(self):
        # indicate that the thread should be stopped
        print "stopping propagation thread"
        self.stoppedPropagationThread = True

class VideoStream:
    def __init__(self, src=0):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()

        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False

    def start(self):
        # start the thread to read frames from the video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        print "starting video stream thread"
        t.start()
        return self

    def update(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return

            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        # return the frame most recently read
        return self.frame

    def stop(self):
        # indicate that the thread should be stopped
        print "stopping video stream thread"
        self.stopped = True
        


sys.stdout = open('/dev/stdout', 'w')

running_network = Network().startPropagationThread()

running_capture = VideoStream(src=0).start() # varies

while(True):
    bgr_img = 0
    bgr_img = running_capture.read()
    img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)

    running_network.feed_full_image(img)
    
    targets = running_network.found_targets_bounding_boxes
    
    while targets:
        t = targets.pop()
        #for t in targets:
        print "found robot on " + str(t[0]) + " " + str(t[1])
        cv2.rectangle(bgr_img,(t[0],t[1]),(t[2],t[3]),(0,0,255),1)

    cv2.imshow('ImageWindow', bgr_img)
    
    k = cv2.waitKey(27)
    if k == 27:
        # stop the threads
        running_capture.stop()
        sleep(1)
        running_network.stopPropagationThread()
        sleep(1)
        cv2.destroyAllWindows()
        sleep(1)
        break

In [5]:
print running_network.robotRadiuses
#running_capture.stop()
#sleep(1)
#running_network.stopPropagationThread()
#sleep(1)
#cv2.destroyAllWindows()

In [8]:
from math import sin, cos, radians
def pixel2scale(height, angle, pixel_y):
    # returns number of meters one pixel width is on the ground
    # F.ex if it returns 2, 10 pixels width is 20 meters on the ground

    # Will return negative values if the pixel does not correspond to a point on the ground

    # height from ground to camera in meters
    # angle from downwards vector to camera center direction
    # pixel y coordinate from top

    #pinhole camera model
    camera_ox = 640/2
    camera_oy = 480/2
    camera_focal = 1360.4/1080 * 480    

    dy = pixel_y-camera_oy
    dz = camera_focal

    sa, ca = sin(radians(angle)), cos(radians(angle))
    dz = ca*dz+sa*dy

    t = height/dz
    return t

def pixel2robotRadius(y):
    return 0.33/pixel2scale(1, 70, y)

def get_ystart():
    smallest_i = None
    for i in range(480):
        if pixel2robotRadius(i) > 0 and smallest_i == None:
            smallest_i = i
    return smallest_i

def get_yend():
    yend = None
    for y in range(ystart, 480):
        if y + pixel2robotRadius(y) < 480:
            yend = y
    return yend
ystart = get_ystart()
yend = get_yend()
ys = [y for y in range(ystart,yend+1)]



running_network.robotRadiuses



[0.01999883471157631,
 0.3300973995709251,
 0.6401959644302738,
 0.9502945292896225,
 1.2603930941489714,
 1.57049165900832,
 1.880590223867669,
 2.190688788727027,
 2.500787353586376,
 2.8108859184457247,
 3.1209844833050737,
 3.431083048164422,
 3.741181613023771,
 4.05128017788312,
 4.361378742742469,
 4.671477307601817,
 4.981575872461166,
 5.291674437320524,
 5.601773002179873,
 5.9118715670392215,
 6.221970131898571,
 6.53206869675792,
 6.842167261617267,
 7.152265826476617,
 7.462364391335965,
 7.772462956195314,
 8.082561521054672,
 8.392660085914022,
 8.70275865077337,
 9.01285721563272,
 9.322955780492068,
 9.633054345351416,
 9.943152910210765,
 10.253251475070115,
 10.563350039929462,
 10.87344860478881,
 11.18354716964817,
 11.493645734507517,
 11.803744299366867,
 12.113842864226216,
 12.423941429085565,
 12.734039993944913,
 13.044138558804262,
 13.354237123663612,
 13.66433568852296,
 13.974434253382318,
 14.284532818241667,
 14.594631383101015,
 14.904729947960364,
 15

In [12]:
ys = [y for y in range(ystart,yend+1)]
rs = [pixel2robotRadius(y) for y in range(ystart,yend+1)]


print len(ys)
print len(rs)



print ys

352
352
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 2

In [58]:
import sys
from math import floor, ceil
from math import sin, cos, radians
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from time import sleep
import cv2
from threading import Thread
%matplotlib inline

#pinhole camera model
camera_ox = 640/2
camera_oy = 480/2
camera_focal = 1360.4/1080 * 480

def pixel2scale(height, angle, pixel_y):
    # returns number of meters one pixel width is on the ground
    # F.ex if it returns 2, 10 pixels width is 20 meters on the ground

    # Will return negative values if the pixel does not correspond to a point on the ground

    # height from ground to camera in meters
    # angle from downwards vector to camera center direction
    # pixel y coordinate from top
    
    dy = pixel_y-camera_oy
    dz = camera_focal

    sa, ca = sin(radians(angle)), cos(radians(angle))
    dz = ca*dz+sa*dy

    t = height/dz
    return t

def pixel2robotRadius(y):
    return 0.33/pixel2scale(1, 70, y)

def get_ystart():
    smallest_i = None
    for i in range(480):
        if pixel2robotRadius(i) > 0 and smallest_i == None:
            smallest_i = i
    return smallest_i

def get_yend():
    yend = None
    for y in range(ystart, 480):
        if y + pixel2robotRadius(y) < 480:
            yend = y
    return yend

class VideoStream:
    def __init__(self, src=0):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()

        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False

    def start(self):
        # start the thread to read frames from the video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        print "starting video stream thread"
        t.start()
        return self

    def update(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return

            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        # return the frame most recently read
        return self.frame

    def stop(self):
        # indicate that the thread should be stopped
        print "stopping video stream thread"
        self.stopped = True

ystart = get_ystart()
yend = get_yend()

print ystart
print yend

running_capture = VideoStream(src=0).start() # varies

sleep(1)

bgr_img = running_capture.read()
img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)

sleep(1)

running_capture.stop()
sleep(1)

# assumtion of the world
ys = [y for y in range(ystart,yend+1)]
robotRadiuses = [pixel2robotRadius(y) for y in range(ystart,yend+1)]
#print robotRadiuses


for y, r in zip(ys,robotRadiuses):
    #if y < 1:
    #    pass
    pixel_width = len(img[0,:])
    x_steps = int(pixel_width/r)
    #print x_steps
    #print floor(x_steps)
    #print type(x_steps)
    #sys.exit()
    #for x in range(x_steps):
    for x in range(0, pixel_width - int(r)):
        #print y
        #print x
        #temp = img[y*r:y*r+r,x*r:x*r+r]
        #print temp.shape
        #center_x = x*r + (x*r+r - x*r)/2
        #center_y = y*r + (y*r+r - y*r)/2
        #portions[(center_x,center_y)] = temp
        
        if run_inference_on_video(img, sess, labels) == "robot":
            x1 = x
            y1 = y
            x2 = int(x+r)
            y2 = int(y+r)
            cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)

#cv2.rectangle(img,(0,200),(150,400),(255,0,0),1)

cv2.imshow('original',bgr_img)
cv2.imshow('with search windows',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#plt.imshow(img)

20
371
starting video stream thread
stopping video stream thread


In [5]:
    portions = []
    y_steps = len(img[:,0])/64
    x_steps = len(img[0,:])/64
    for y in range(y_steps):
        for x in range(x_steps):
            #print y
            #print x
            temp = img[y*64:y*64+64,x*64:x*64+64]
            #print temp.shape
            portions.append(temp)
            #cv2.imshow('ImageWindow', temp)
    
    if False:
        plt.ion()
        for i in range(len(portions)):   
            plt.subplot(y_steps,x_steps,i+1)
            take = portions[i]
            plt.imshow(take)
        plt.show()
        plt.pause(0.05)
    if False:
        for i in range(len(portions)):
            take = portions[i]
            #norm = take/255.0
            #print run_inference_on_video(take, sess, labels)

    #cv2.imwrite("/home/ascend/Desktop/temp.jpg", res)stream thread"
        self.stopped = True

In [1]:
import sys
sys.stdout = open('/dev/stdout', 'w')
running_capture = VideoStream(src=1).start() # varies

# assumtion of the world
ys = [y for y in range(ystart,yend+1)]
robotRadiuses = [pixel2robotRadius(y) for y in range(ystart,yend+1)]

while(True):
    bgr_img = running_capture.read()
    
    img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    portions = {}
    for y, r in zip(ys,robotRadiuses):
        x_steps = len(img[0,:])/r
        for x in range(x_steps):
            #print y
            #print x
            #temp = img[y*r:y*r+r,x*r:x*r+r]
            #print temp.shape
            #center_x = x*r + (x*r+r - x*r)/2
            #center_y = y*r + (y*r+r - y*r)/2
            #portions[(center_x,center_y)] = temp
            #cv2.imshow('ImageWindow', temp)
            pass
    
    #img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    #cv2.rectangle(bgr_img,(0,0),(200,100),(0,0,255),3)
    cv2.imshow('ImageWindow', bgr_img)
    
    k = cv2.waitKey(27)
    if k == 27:
        # stop the threads
        running_capture.stop()
        cv2.destroyAllWindows()
        sleep(0.1)
        break

NameError: name 'VideoStream' is not defined

In [310]:
running_capture = VideoStream(src=0).start() # varies

running_capture.stop()

cv2.destroyAllWindows()

def run_inference_on_image():
    answer = None

    if not tf.gfile.Exists(self.imagePath):
        tf.logging.fatal('File does not exist %s', self.imagePath)
        return answer

    image_data = tf.gfile.FastGFile(self.imagePath, 'rb').read()

    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:

        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions)

        top_k = predictions.argsort()[-5:][::-1]  # Getting top 5 predictions
        f = open(self.labelsFullPath, 'rb')
        lines = f.readlines()
        labels = [str(w).replace("\n", "") for w in lines]
        for node_id in top_k:
            human_string = labels[node_id]
            score = predictions[node_id]
            print('%s (score = %.5f)' % (human_string, score))

        answer = labels[top_k[0]]
        return answer